In [8]:
import pandas as pd
import numpy as np
from datetime import datetime

In [9]:
df = pd.read_csv('Data/Camp_Market.csv', sep=',')

In [10]:
print(pd.get_option("display.max_rows"))
print(pd.get_option("display.max_columns"))

60
20


In [11]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce').dt.strftime('%d/%m/%Y')

/var/folders/4n/0hqdswrx4pn48s5909d75wzr0000gn/T/ipykernel_13430/1408691525.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], errors='coerce').dt.strftime('%d/%m/%Y')


In [12]:
cols_to_check = [
    'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome', 'Teenhome',
    'Dt_Customer', 'Recency', 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts',
    'MntSweetProducts', 'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
    'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'
]

duplicate_mask = df.duplicated(subset=cols_to_check, keep='first')

df = df.drop_duplicates(subset=cols_to_check, keep='first').reset_index(drop=True)
df.to_csv("Data/Camp_Market_Cleaned.csv", index=False)

In [13]:
df['Marital_Status'] = df['Marital_Status'].replace({
    'Alone': 'Single',
    'Absurd': 'Single',
    'YOLO': 'Single',
    'Divorced': 'Single',
    'Widow': 'Single',
    'Married': 'Couple',
    'Together': 'Couple'
})

df['Education'] = df['Education'].replace({
    'Basic': 'Undergrad',
    '2n Cycle': 'Undergrad',
    'Graduation': 'Graduate',
    'Master': 'Postgrad',
    'PhD': 'Postgrad'
})

df.rename(columns={'Dt_Customer': 'Customer_For'}, inplace=True)
df.rename(columns={'Recency': 'Last_Purchase'}, inplace=True)

df['Childrens'] = df['Kidhome'] + df['Teenhome']
df['Spent'] = df[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']].sum(axis=1)
df['Family_Size'] = df['Kidhome'] + df['Teenhome'] + df['Marital_Status'].map({'Single': 1, 'Couple': 2})
df['Age'] = datetime.now().year - df['Year_Birth']

df = df.drop(columns=['Kidhome', 'Teenhome'])  


In [14]:
missing_idx = df['Income'].isna()  

def impute_income(row, df):
    if pd.notna(row['Income']):
        return row['Income'] 
    
    mask = (
        (df['Year_Birth'] == row['Year_Birth']) &
        (df['Education'] == row['Education']) &
        (df['Marital_Status'] == row['Marital_Status']) &
        (df['Income'].notna())
    )
    
    group_values = df.loc[mask, 'Income']
    
    if len(group_values) > 0:
        return round(group_values.mean()) 
    else:
        return np.nan

df['Income'] = df.apply(lambda row: impute_income(row, df), axis=1).astype('Int64')


In [15]:
display(df[missing_idx & df['Income'].notna()])

,ID,Year_Birth,Education,Marital_Status,Income,Customer_For,Last_Purchase,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Childrens,Spent,Family_Size,Age
10,1994,1983,Graduate,Couple,42308,15/11/2013,11,5,5,6,...,0,0,0,3,11,0,1,19,3,42
27,5255,1986,Graduate,Single,31159,20/02/2013,19,5,1,3,...,0,0,0,3,11,0,1,637,2,39
43,7281,1959,Postgrad,Single,53411,05/11/2013,80,81,11,50,...,0,0,0,3,11,0,0,186,1,66
48,7244,1951,Graduate,Single,53472,01/01/2014,96,48,5,48,...,0,0,0,3,11,0,3,124,4,74
58,8557,1982,Graduate,Single,45618,17/06/2013,57,11,3,22,...,0,0,0,3,11,0,1,46,2,43
71,10629,1973,Undergrad,Couple,34210,14/09/2012,25,25,3,43,...,0,0,0,3,11,0,1,109,3,52
89,8996,1957,Postgrad,Couple,55337,19/11/2012,4,230,42,192,...,0,0,0,3,11,0,3,603,5,68
90,9235,1957,Graduate,Single,60037,27/05/2014,45,7,0,8,...,0,0,0,3,11,0,2,18,3,68
91,5798,1973,Postgrad,Couple,54859,23/11/2013,87,445,37,359,...,0,0,0,3,11,0,0,985,2,52
127,8268,1961,Postgrad,Couple,60176,11/07/2013,23,352,0,27,...,0,0,0,3,11,0,1,404,3,64


In [16]:
display(df)

,ID,Year_Birth,Education,Marital_Status,Income,Customer_For,Last_Purchase,MntWines,MntFruits,MntMeatProducts,...,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response,Childrens,Spent,Family_Size,Age
0,5524,1957,Graduate,Single,58138,04/09/2012,58,635,88,546,...,0,0,0,3,11,1,0,1617,1,68
1,2174,1954,Graduate,Single,46344,08/03/2014,38,11,1,6,...,0,0,0,3,11,0,2,27,3,71
2,4141,1965,Graduate,Couple,71613,21/08/2013,26,426,49,127,...,0,0,0,3,11,0,0,776,2,60
3,6182,1984,Graduate,Couple,26646,10/02/2014,26,11,4,20,...,0,0,0,3,11,0,1,53,3,41
4,5324,1981,Postgrad,Couple,58293,19/01/2014,94,173,43,118,...,0,0,0,3,11,0,1,422,3,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2034,9432,1977,Graduate,Couple,666666,02/06/2013,23,9,14,18,...,0,0,0,3,11,0,1,62,3,48
2035,10870,1967,Graduate,Couple,61223,13/06/2013,46,709,43,182,...,0,0,0,3,11,0,1,1341,3,58
2036,7270,1981,Graduate,Single,56981,25/01/2014,91,908,48,217,...,0,0,0,3,11,0,0,1241,1,44
2037,8235,1956,Postgrad,Couple,69245,24/01/2014,8,428,30,214,...,0,0,0,3,11,0,1,843,3,69


In [17]:
df.to_csv("Data/Camp_Market_Cleaned.csv", index=False)